In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import leastsq
from scipy import ndimage
from func import *
from astropy.table import Table, hstack
import os
import glob
from astropy.table import Table, vstack, join
import pandas as pd

In [2]:
def tab(obj,par):
    column_names={'Hb_x':0,'Hb_y':1,'OIII_br_x':2,'OIII_br_y':3,'OIII_nr_x':4,'OIII_nr_y':5,'off_OIII_br_pix_x':6,'off_OIII_br_pix_y':7,'off_OIII_nr_pix_x':8,'off_OIII_nr_pix_y':9,'off_OIII_br_arc_x':10,'off_OIII_br_arc_y':11,'off_OIII_nr_arc_x':12,'off_OIII_nr_arc_y':13,'off_wing_arc':14,'off_wing_parsec':15}
    columns=[]
    for key in column_names.keys():
        columns.append(fits.Column(name=key,format='E',array=[par[column_names[key]]]))
    coldefs = fits.ColDefs(columns)
    hdu = fits.BinTableHDU.from_columns(coldefs)
    hdu.writeto('%s_center_table.fits'%(obj),overwrite=True)   

def source_tab(obj,z,d_a):
    a1 = np.array(['%s'%(obj)])
    a2 = np.array([z])
    a3 = np.array([d_a])
    hdu = fits.BinTableHDU.from_columns([fits.Column(name='Source', format='20A', array=a1),
          fits.Column(name='z',format='E',array=a2),fits.Column(name='d_a',format='E',array=a3)])
    hdu.writeto('Source_name_%s.fits'%(obj),overwrite=True)  

def create_center_table(obj):
    t1 = Table.read('Source_name_%s.fits'%(obj),format='fits')
    t2 = Table.read('%s_center_table.fits'%(obj),format='fits')
    new = hstack([t1, t2])
    new.write('%s_Moffat_centroids_with_source.fits'%(obj),overwrite=True)
    
def vel_table(obj):
    t1 = Table.read('Source_name_%s.fits'%(obj),format='fits')
    t2 = Table.read('%s_central_fit.fits'%(obj),format='fits')
    new = hstack([t1, t2])
    new.write('%s_central_tab.fits'%(obj),overwrite=True)

def par_table(obj):
    t1 = Table.read('Source_name_%s.fits'%(obj),format='fits')
    t2 = Table.read('%s_par_table.fits'%(obj),format='fits')
    new = hstack([t1, t2])
    new.write('%s_parameter.fits'%(obj),overwrite=True)
    
def nonspectro_table(obj):
    t1 = Table.read('Source_name_%s.fits'%(obj),format='fits')
    t2 = Table.read('%s_non_spectro_central_fit.fits'%(obj),format='fits')
    new = hstack([t1, t2])
    new.write('%s_central_nonspectab.fits'%(obj),overwrite=True)
    
def nonspectro_BH_table(obj):
    t1 = Table.read('Source_name_%s.fits'%(obj),format='fits')
    t2 = Table.read('%s_par_table_nonspec.fits'%(obj),format='fits')
    new = hstack([t1, t2])
    new.write('%s_par_table_nonspecBH.fits'%(obj),overwrite=True)
    
def spectro_BH_table(obj):
    t1 = Table.read('Source_name_%s.fits'%(obj),format='fits')
    t2 = Table.read('%s_par_table_spec.fits'%(obj),format='fits')
    new = hstack([t1, t2])
    new.write('%s_par_table_specBH.fits'%(obj),overwrite=True)

def flux_table(obj):
    t1 = Table.read('Source_name_%s.fits'%(obj),format='fits')
    t2 = Table.read('%s_compare_flux_table.fits'%(obj),format='fits')
    new = hstack([t1, t2])
    new.write('%s_fluxtab.fits'%(obj),overwrite=True)

In [3]:
def algorithm_script(obj,z,d_a,prefix_path_cube="/home/mainak/xdata/ftp.hidrive.strato.com/users/login-carsftp"):
    (cont_cube,cont_err,cont_wave,cont_header) = loadCube('%s/MUSE/%s/%s.cont_model.fits'%(prefix_path_cube,obj,obj))
    muse_sampling_size = sampling_size(cont_cube)
   
    (Hb_x,Hb_y,OIII_br_x,OIII_br_y,OIII_nr_x,OIII_nr_y) = centers(obj)
    (off_OIII_br_pix,off_OIII_nr_pix,off_OIII_br_arc,off_OIII_nr_arc) = offset(Hb_x,Hb_y,OIII_br_x,OIII_br_y,OIII_nr_x,OIII_nr_y,muse_sampling_size)
    (off_OIII_br_pix_x,off_OIII_br_pix_y) = off_OIII_br_pix
    (off_OIII_nr_pix_x,off_OIII_nr_pix_y) = off_OIII_nr_pix
    (off_OIII_br_arc_x,off_OIII_br_arc_y) = off_OIII_br_arc
    (off_OIII_nr_arc_x,off_OIII_nr_arc_y) = off_OIII_nr_arc
    off_wing_arc = np.sqrt((off_OIII_br_arc_x)**2 + (off_OIII_br_arc_y)**2)
    off_wing_parsec = 4.848*d_a*off_wing_arc
    #print '%s'%(obj),off_wing_arc,d_a,off_wing_parsec
    
    par = [Hb_x,Hb_y,OIII_br_x,OIII_br_y,OIII_nr_x,OIII_nr_y,off_OIII_br_pix_x,off_OIII_br_pix_y,off_OIII_nr_pix_x,off_OIII_nr_pix_y,off_OIII_br_arc_x,off_OIII_br_arc_y,off_OIII_nr_arc_x,off_OIII_nr_arc_y,off_wing_arc,off_wing_parsec]
    tab(obj,par)
    source_tab(obj,z,d_a)
    create_center_table(obj)
    vel_table(obj)
    par_table(obj)
    nonspectro_table(obj)
    nonspectro_BH_table(obj)
    spectro_BH_table(obj)
    flux_table(obj)

In [4]:
z = {"HE0021-1819":0.053197,"HE0040-1105":0.041692 #,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":0.054341,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143,"HE0253-1641":0.031588
    ,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009,"HE0433-1028":0.035550
    ,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986,"HE1029-1831":0.040261
    ,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1330-1013":0.022145,"HE1353-1917":0.035021
    ,"HE1417-0909":0.044,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}


d_a = {"HE0021-1819":213.8,"HE0040-1105":170.9 #,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":218.1,"HE0224-2834":238.5,"HE0227-0913":69.07,"HE0232-0900":175.5,"HE0253-1641":130.2
    ,"HE0345+0056":127.9,"HE0351+0240":147.7,"HE0412-0803":156.1,"HE0429-0247":171.1,"HE0433-1028":145.9
    ,"HE0853+0102":209.3,"HE0934+0119":203.0,"HE1011-0403":233.0,"HE1017-0305":201.7,"HE1029-1831":164.3
    ,"HE1107-0813":231.8,"HE1108-2813":99.91,"HE1126-0407":246.5,"HE1330-1013":92.344,"HE1353-1917":143.8
    ,"HE1417-0909":178.8,"HE2211-3903":162.2,"HE2222-0026":236.0,"HE2233+0124":226.2,"HE2302-0857":189.8}

#z_remaining = {"HE2128-0221":0.05248,"HE1248-1356":0.01465}

objs = z.keys()

for obj in objs:
    algorithm_script(obj,z[obj],d_a[obj])

In [5]:
def master_table(input_filename,output_filename):
    infiles = sorted(glob.glob(input_filename)) # './*source.fits'
    infiles
    tabledict = {}

    for i, file in enumerate(infiles):
        hdulist = fits.open(file)
        bintab = hdulist[1].data
        table = Table(bintab)
        name = table['Source'][0]  
        tabledict[name] = table

    tab = vstack(list(tabledict.values()))
    df = tab.to_pandas()
    df = df.sort_values(by='Source', ascending=True)
    dtab = Table.from_pandas(df)
    dtab.write(output_filename, format='fits', overwrite=True)

In [6]:
#def table_algorithmic_script(input_filename,output_filename):
master_table(input_filename='./*with_source.fits',output_filename='final_table.fits')
master_table(input_filename='./*tab.fits',output_filename='central_table.fits')
master_table(input_filename='./*parameter.fits',output_filename='par_table.fits')
master_table(input_filename='./*nonspectab.fits',output_filename='nonspectro_table.fits')
master_table(input_filename='./*nonspecBH.fits',output_filename='nonspectro_BH_table.fits')
master_table(input_filename='./*specBH.fits',output_filename='spectro_BH_table.fits')
master_table(input_filename='./*fluxtab.fits',output_filename='flux_table.fits')

In [7]:
#Merging the tables from Julia
t1 = Table.read('nuclear_fits_20160811/good/em_fits_FeII_PL.fits',format='fits')
t2 = Table.read('nuclear_fits_20160811/problematic/em_fits_FeII_PL.fits',format='fits')
tab = vstack([t1, t2])
df = tab.to_pandas()
df = df.sort_values(by='names', ascending=True)
dtab = Table.from_pandas(df) 
fin_tab = dtab[[0,1,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23,24,25,27,28,29,30]]
fin_tab.write('BH_par.fits',overwrite=True)   